In [1]:
import nb_setup
import importlib

importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [29]:
%time
import time
from hyperliquid.info import Info
from hyperliquid.utils import constants as hl_constants
import utils.now_timestamp as now_timestamp 
import models.candles as candles
import importlib
importlib.reload(candles)
importlib.reload(now_timestamp)

info = Info(hl_constants.MAINNET_API_URL, skip_ws=True)
def fetch_coins_by_names(names):
    start = now_timestamp.get_timestamp_from_today_midnight_utc(days=-30)
    end = now_timestamp.get_timestamp_from_today_midnight_utc(days=0)
    error_coins = []
    for coin in names:
        try:
            time.sleep(0.2)
            print(coin)
            result = info.candles_snapshot(coin, '15m', start, end)
            _candles = candles.CandleList.from_response(result)
            _candles.to_csv()
        except Exception as e:
            print(f"coin {coin}: not fetched")
            print(e)
            error_coins.append(coin)
            continue

fetch_coins_by_names(["BTC"])

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 13.1 µs
BTC


In [21]:
import pandas as pd
import models.backtesting.backtesting as backtesting
import models.backtesting.BollingerBandsReEntryStrategy as bollingerBandsReEntryStrategy

importlib.reload(bollingerBandsReEntryStrategy)
importlib.reload(backtesting)
BackTesting = backtesting.BackTesting
EXPECTED_DTYPES = backtesting.EXPECTED_DTYPES


df = pd.read_csv("data/BTC.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.astype(EXPECTED_DTYPES)

df.shape
df.describe()

,datetime,number_of_trades,close,high,low,open,volume,tic,toc
count,2881,2881.000000,2881.000000,2881.000000,2881.000000,2881.00000,2881.000000,2.881000e+03,2.881000e+03
mean,2024-04-08 23:59:59.999999488,1127.011107,67457.287747,67594.065602,67310.290524,67457.89101,70.961191,1.712621e+12,1.712622e+12
min,2024-03-25 00:00:00,186.000000,59663.000000,60336.000000,59571.000000,59650.00000,4.390790,1.711325e+12,1.711326e+12
25%,2024-04-01 12:00:00,610.000000,65112.000000,65307.000000,64977.000000,65117.00000,28.044190,1.711973e+12,1.711974e+12
50%,2024-04-09 00:00:00,896.000000,67737.000000,67839.000000,67612.000000,67736.00000,47.681010,1.712621e+12,1.712622e+12
75%,2024-04-16 12:00:00,1401.000000,70016.000000,70121.000000,69897.000000,70020.00000,88.279050,1.713269e+12,1.713270e+12
max,2024-04-24 00:00:00,10788.000000,72764.000000,72836.000000,72444.000000,72765.00000,1107.488910,1.713917e+12,1.713918e+12
std,NaN,786.325011,2839.869358,2813.730097,2870.132055,2840.03028,71.834789,7.486356e+08,7.486356e+08


In [15]:
# Create a strategy
bt = BackTesting()
INITAL_CAPITAL = 200
BUY_EQUITY = 10
LEVERAGE = 1
STOP_LOSS_PERCENTAGE = 0 # 0 - 1
K = 2
bt.run_backtesting(
    df,
    strategy=bollingerBandsReEntryStrategy.BollingerBandsReEntryStrategy(
        initial_capital=INITAL_CAPITAL,
        buy_equity=BUY_EQUITY,
        leverage=LEVERAGE,
        k=K,
        stop_loss_percentage=STOP_LOSS_PERCENTAGE
    ),
    # play_mode=True,
)

Validating data...
Data validated successfully.
Start Running backtesting...
Data shape: (2881, 13)
Strategy: <models.backtesting.BollingerBandsReEntryStrategy.BollingerBandsReEntryStrategy object at 0x126272f10>
Processing row 0...
Portfolio Value:  (1, 6) : 200
Processing row 1...
shape(self.data):  (2, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (2, 6) : 200
Processing row 2...
shape(self.data):  (3, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (3, 6) : 200
Processing row 3...
shape(self.data):  (4, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (4, 6) : 200
Processing row 4...
shape(self.data):  (5, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (5, 6) : 200
Processing row 5...
shape(self.data):  (6, 13) states:  nan nan
No valid trade condition met.
Portfolio Value:  (6, 6) : 200
Processing row 6...
shape(self.data):  (7, 13) states:  nan nan
No valid trade condition met.
Portfolio Value: 

/Users/falcon/Developments/act/backtesting-playground/trading/models/backtesting/BollingerBandsReEntryStrategy.py:155: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/falcon/Developments/act/backtesting-playground/trading/models/backtesting/portfolio.py:249: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/falcon/Developments/act/backtesting-playground/trading/models/backtesting/BollingerBandsReEntryStrategy.py:282: FutureWarning:

The behavior of DataFrame concatenation with empty or a

shape(self.data):  (55, 13) states:  BollingerBandsState.WITHIN_BANDS BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (55, 6) : 200.0423421422755200236879118
Processing row 55...
shape(self.data):  (56, 13) states:  BollingerBandsState.ABOVE_UPPER_BAND BollingerBandsState.WITHIN_BANDS
No valid trade condition met.
Portfolio Value:  (56, 6) : 199.9181286549707602339181287
Processing row 56...
shape(self.data):  (57, 13) states:  BollingerBandsState.ABOVE_UPPER_BAND BollingerBandsState.ABOVE_UPPER_BAND
No valid trade condition met.
Portfolio Value:  (57, 6) : 199.8365534088385520763935154
Processing row 57...
shape(self.data):  (58, 13) states:  BollingerBandsState.ABOVE_UPPER_BAND BollingerBandsState.ABOVE_UPPER_BAND
No valid trade condition met.
Portfolio Value:  (58, 6) : 199.7438744540676585979717226
Processing row 58...
shape(self.data):  (59, 13) states:  BollingerBandsState.ABOVE_UPPER_BAND BollingerBandsState.ABOVE_UPPER_BAND
No valid trade condit

In [16]:
bt.strategy.portfolio.holdings

{'BTC': [Symbol: BTC, Entry Price: 66068, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001513592056668886601683114367,
  Symbol: BTC, Entry Price: 66284, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001508659706716553014302094020]}

In [17]:
# display(bt.strategy.history)
# display(bt.strategy.portfolio.portfolio_history)
bt.strategy.portfolio.info()

Start Time:  2024-03-25 00:00:00
End Time:  2024-04-24 00:00:00
Cash:  194.6639905084558238527709968
Holdings: 
{'BTC': [Symbol: BTC, Entry Price: 66068, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001513592056668886601683114367,
         Symbol: BTC, Entry Price: 66284, Equity: 10, Side: Side.LONG, Leverage: 1, Size: 0.0001508659706716553014302094020]}
Evaluation:  214.7921872526028516952324846
PNL:  14.7921872526028516952324846
PNL Percentage:  7.396093626301425847616242300
Max DrawDown Percentage:  -6.108542627496892886831013320
Max Return Percentage:  10.10111974565922021101121326


In [18]:
from settings import DATA_DIR
name = f"{INITAL_CAPITAL}-{BUY_EQUITY}-{LEVERAGE}x-{K}std-{STOP_LOSS_PERCENTAGE}SL_BTC_BollingerBandsReEntryStrategy"
bt.strategy.history.to_csv(DATA_DIR / f"./{name}-history.csv")
bt.strategy.portfolio.portfolio_history.to_csv(
    DATA_DIR / f"./{name}-portfolio-history.csv"
)

In [22]:
bt.strategy.plot_history()

In [23]:
bt.strategy.plot_performance()